# Run simulations with O'Regan 2022 parameter set (LG M50)

In this notebook we show an example on how to run the DFN model with the O'Regan 2022 parameter set for the LG M50 cell. Because of the concentration dependent diffusion coefficient, we need to customise the mesh so the simulations converge.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

We first initialise the model and define the parameter set and the solver:

In [ ]:
# DFN + lumped thermal
options = {"thermal": "lumped", "dimensionality": 0, "cell geometry": "arbitrary"}
model = pybamm.lithium_ion.DFN(options=options)

# O'Regan 2022 parameter set
param = pybamm.ParameterValues("ORegan2022")

# Choose CasADI fast (we do a short discharge so there are no events, if events are needed choose "fast with events")
solver = pybamm.CasadiSolver(mode="fast")

Now we define the new mesh. Because the particle diffusivity depends on concentration and get drop very abruptly, we set a fine mesh (40 points in each particle). Additionally, given that the larger gradients occur towards the particle surface, we choose an exponential mesh weighted towards the `right` (i.e. the particle surface). This generates a non-uniform mesh with a finer grid towards the particle surface and a coarser grid towards the particle centre.

In [ ]:
var_pts = {"x_n": 30, "x_s": 30, "x_p": 30, "r_n": 40, "r_p": 40}

submesh_types = model.default_submesh_types
submesh_types["negative particle"] = pybamm.MeshGenerator(
    pybamm.Exponential1DSubMesh, submesh_params={"side": "right"}
)
submesh_types["positive particle"] = pybamm.MeshGenerator(
    pybamm.Exponential1DSubMesh, submesh_params={"side": "right"}
)

# Define the simulation
sim = pybamm.Simulation(
    model,
    parameter_values=param,
    C_rate=1,
    solver=solver,
    var_pts=var_pts,
    submesh_types=submesh_types,
)

Finally, we can solve the simulation and plot the results. Note that because the nonlinear diffusion and the fine mesh, if we want to solve for longer time, such as full discharge or other experiments, the simulations might take a few minutes to run.

In [ ]:
sim.solve(
    [0, 10]
)  # solving time kept short for testing purposes, feel free to extend it
sim.plot()

In [ ]:
pybamm.print_citations()